We will explore the relationship between two variables, which are whether an individual owns a house and whether an individual has a loan, to the propensity for term deposit purchases by these individuals.

In [1]:
import pandas as pd

In [2]:
file_url = r'https://raw.githubusercontent.com/sedeba19/Chapter-3/main/data/bank-full.csv'

In [3]:
# Read the bank data
bank_df = pd.read_csv(file_url,
                      sep = ';')

In [4]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
# Relationship between housing and propensity for term deposit
bank_df.groupby(['housing', 'y'])['y'].agg(house_total = 'count').reset_index()

,housing,y,house_total
0,no,no,16727
1,no,yes,3354
2,yes,no,23195
3,yes,yes,1935


In [6]:
#Relationship between having a loan and propensity for term deposits
bank_df.groupby(['loan','y'])['y'].agg(loan_total ='count').reset_index()

,loan,y,loan_total
0,no,no,33162
1,no,yes,4805
2,yes,no,6760
3,yes,yes,484


In [7]:
bank_df.shape

(45211, 17)

In [8]:
# Taking the quantiles for 25%, 50% and 75% of the balance data
import numpy as np
np.quantile(bank_df['balance'], [0.25, 0.5, 0.75])

array([  72.,  448., 1428.])

In [9]:
# Convert the numerical values of bank balances into categorical values
bank_df['balanceClass'] = 'Quant1'
bank_df.loc[(bank_df['balance'] > 72) & (bank_df['balance'] < 448),
            'balanceClass'] = 'Quant2'
bank_df.loc[(bank_df['balance'] > 448) & (bank_df['balance'] < 1428), 
              'balanceClass'] = 'Quant3'
bank_df.loc[bank_df['balance'] > 1428, 
             'balanceClass'] = 'Quant4'
bank_df.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,balanceClass
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,Quant4
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,Quant1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,Quant1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,Quant4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,Quant1


In [10]:
# Calculate the custoimers under each quantile
balance_total = bank_df.groupby(['balanceClass'])['y'].agg(balance_total = 'count').reset_index()
balance_total

,balanceClass,balance_total
0,Quant1,11340
1,Quant2,11275
2,Quant3,11299
3,Quant4,11297


In [11]:
# Calculate the total customers categorized as per quantile and propensity classification
balance_propensity = bank_df.groupby(['balanceClass', 'y'])['y'].agg(balance_cat = 'count').reset_index()
balance_propensity

,balanceClass,y,balance_cat
0,Quant1,no,10517
1,Quant1,yes,823
2,Quant2,no,10049
3,Quant2,yes,1226
4,Quant3,no,9884
5,Quant3,yes,1415
6,Quant4,no,9472
7,Quant4,yes,1825


In [12]:
# Merge both DataFrame
balance_merge = pd.merge(balance_propensity,
                         balance_total,
                         on = ['balanceClass'])
balance_merge

,balanceClass,y,balance_cat,balance_total
0,Quant1,no,10517,11340
1,Quant1,yes,823,11340
2,Quant2,no,10049,11275
3,Quant2,yes,1226,11275
4,Quant3,no,9884,11299
5,Quant3,yes,1415,11299
6,Quant4,no,9472,11297
7,Quant4,yes,1825,11297


In [13]:
balance_merge['category_propensity'] = (balance_merge['balance_cat']/balance_merge['balance_total'])*100
balance_merge

,balanceClass,y,balance_cat,balance_total,category_propensity
0,Quant1,no,10517,11340,92.742504
1,Quant1,yes,823,11340,7.257496
2,Quant2,no,10049,11275,89.126386
3,Quant2,yes,1226,11275,10.873614
4,Quant3,no,9884,11299,87.476768
5,Quant3,yes,1415,11299,12.523232
6,Quant4,no,9472,11297,83.845269
7,Quant4,yes,1825,11297,16.154731
